# Tổng quan phần thu thập dữ liệu
1. Mục tiêu
- Hiện nay có rất nhiều nhà tuyển dụng đăng thông tin tuyển dụng trên nhiều trang web như : https://vieclam24h.vn/, https://www.topcv.vn/viec-lam, https://careerbuilder.vn/tim-viec-lam.html, https://www.vieclamtot.com/viec-lam
- Do đó để dự đoán xu hướng công việc cũng như tìm hiểu thị trường việc làm, chúng ta sẽ chọn thu thập dữ liệu từ các trang web [Viêc làm 24h](https://vieclam24h.vn/).
2. Công cụ sử dụng
- Trong phần này, ta sử dụng những thư viện được python hỗ trợ như: request_html để gửi yêu cầu lấy trang web về.
- BeautifulSoup để phân tích cấu trúc của trang web. Sau đó lấy những trường cần thiết.
3. Các bước thu thập dữ liệu
- Ban đầu trang chủ trang web có một các thông tin về các công việc mới nhất, các công việc được đăng gần đây nhất. Ta sẽ lấy các link của các công việc này. Sau đó đến đường dẫn các công việc này để lấy thông tin chi tiết về công việc. Chúng em sẽ có 2 bước thu thập dữ liệu:
    - Step 1: Thu thập dữ liệu từ trang chủ sau đó lưu các đường link vào 'job_pool.csv'
    - Thu thập dữ liệu từ các trang chi tiết và lưu tất cả công việc vào một dataset có tên là 'raw_dataset.csv'

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
from os import path
class Path():
  def __init__(self) -> None:
      job_pool_path = 'job_pool.csv'
      raw_dataset_path = 'raw_dataset.csv'

path = Path()

In [ ]:
# !pip install requests-html --quiet
# !pip install bs4 --quiet
# !pip install --upgrade lxml_html_clean --quiet

In [ ]:
# import những  thư viện cần thiết
import pandas as pd
import numpy as np
from requests_html import HTML
from requests.exceptions import RequestException
from requests_html import HTMLSession
from bs4 import BeautifulSoup
import math
import time

In [ ]:
def get_content(url, headers, max_attempts=5):
    session = HTMLSession()
    for attempt in range(max_attempts):
        try:
            response = session.get(url, headers=headers)
            if response.status_code == 200:
                return response
            else:
                print(f'Attempt {attempt+1} failed with status: {response.status_code}')
                
        except RequestException as e:
            print(f'Attempt {attempt+1} failed due to: {str(e)}')
            
        time.sleep(10)
    return None

In [ ]:
BASE_URL = 'https://vieclam24h.vn'

headers = {
   'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'
}

In [ ]:
# url = 'https://vieclam24h.vn/tim-kiem-viec-lam-nhanh?degree_requirement[]={}&experience_range={}&occupation_ids[]={}&working_method[]={}'
url = 'https://vieclam24h.vn/tim-kiem-viec-lam-nhanh?occupation_ids[]={}&working_method[]={}'
# degree_requirements = 8
# experience_range = 8
occupation_ids = 53
working_methods = 6

## Step 0: Mô phỏng trên 1 test case

In [ ]:
jobs_list = []
# test
new_url = 'https://vieclam24h.vn/tim-kiem-viec-lam-nhanh?degree_requirement[]=1&experience_range=3&occupation_ids[]=1&working_method[]=1'
page_content = get_content(new_url, headers)
# Lưu page content vào file
# with open('page_content.html', 'w') as f:
#     f.write(page_content.text)

# Lấy số lượng việc làm
try:
    soup = BeautifulSoup(page_content.text, 'html.parser')
    buffer1 = soup.find('div', class_='relative lg:text-2xl text-xl box-border lg:leading-10 mb-4 m-auto lg:w-full w-[100%] px-0')
    strong_tag = buffer1.find('strong')
    total_jobs = int(strong_tag.text)
except:
    total_jobs = 0


print('Total jobs:', total_jobs)

n_pages = min(51, math.ceil(total_jobs/30) + 1)


# for page in range(1, n_pages + 1):
for page in range(1, 2):
    page_url = new_url + f'&page={page}'


    page_content = get_content(page_url, headers)
    try:
        soup = BeautifulSoup(page_content.text, 'html.parser')

        containers = soup.find_all('div', class_='relative lg:h-[115px] w-full flex rounded-sm lg:mb-3 mb-2 lg:hover:shadow-md')
    except:
        containers = []
    for container in containers:
        dict_job = {}

        buffer = BASE_URL + container.find('a', href=True)['href']
        dict_job['Liên kết'] = buffer if buffer else None

        buffer = container.find('div', class_='relative lg:w-full w-11/12 flex items-start flex-1 overflow-hidden pr-2 lg:pr-8')
        dict_job['Tên công việc'] = buffer.text if buffer else None


        buffer = container.find('span', class_='text-se-neutral-80 text-14 whitespace-nowrap font-medium')
        dict_job['Mức lương'] = buffer.text if buffer else None

        buffer = container.find('span', class_='text-se-neutral-80 whitespace-nowrap text-14')
        dict_job['Khu vực tuyển'] = buffer.text if buffer else None

        jobs_list.append(dict_job)
job_pd = pd.DataFrame(jobs_list)
job_pd

In [ ]:
job_deatail_list = []
for i, job_url in enumerate(job_pd['Liên kết'].head(5)):


    job_content = get_content(job_url, headers=headers)
    if (job_content != None):
        soup = BeautifulSoup(job_content.text, 'html.parser')

    # # Lưu page content vào file
    # with open('job_content.html', 'w') as f:
    #     f.write(job_content.text)

    dict_job = {}

    dict_job['Liên kết'] = job_url

    overview = soup.find('div', class_='md:ml-7 w-full')
    try:
        buffer = overview.find('h2', class_='font-normal text-16 text-se-neutral-64 mb-4')
        dict_job['Tên công ty'] = buffer.text if buffer else None
    except:
        dict_job['Tên công ty'] = None
    try:
        buffer = overview.find('h1', class_='font-semibold text-18 md:text-24 leading-snug')
        dict_job['Tên công việc'] = buffer.text if buffer else None
    except:
        dict_job['Tên công việc'] = None

    date_view = overview.find_all('span', class_='font-medium pr-1')
    try:
        dict_job['Ngày cập nhật'] = date_view[0].find_next_sibling('span').text if date_view[0] else None
    except:
        dict_job['Ngày cập nhật'] = None

    try:
        dict_job['Lượt xem'] = date_view[1].find_next_sibling('span').text if date_view[1] else None
    except:
        dict_job['Lượt xem'] = None

    try:
        buffer = overview.find('p', class_='font-semibold text-14 text-[#8B5CF6]')
        dict_job['Mức lương'] = buffer.text if buffer else None
    except:
        dict_job['Mức lương'] = None

    try:
        buffer = overview.find_all('a', class_ ="hover:text-se-accent")
        dict_job['Khu vực tuyển'] = ' '.join([element.text for element in buffer]) if buffer else None
    except:
        dict_job['Khu vực tuyển'] = None

    try:
        buffer = soup.find('p', string="Yêu cầu giới tính")
        dict_job['Yêu cầu giới tính'] = buffer.find_next_sibling('p').get_text() if buffer else None
    except:
        dict_job['Yêu cầu giới tính'] = None

    try:
        buffer = soup.find('p', string="Cấp bậc")
        dict_job['Cấp bậc'] = buffer.find_next_sibling('p').get_text() if buffer else None
    except:
        dict_job['Cấp bậc'] = None

    try:
        buffer = soup.find('p', string="Thời gian thử việc")
        dict_job['Thời gian thử việc'] = buffer.find_next_sibling('p').get_text() if buffer else None
    except:
        dict_job['Thời gian thử việc'] = None

    try:
        buffer = soup.find('p', string="Số lượng tuyển")
        dict_job['Số lượng tuyển'] = buffer.find_next_sibling('p').get_text() if buffer else None
    except:
        dict_job['Số lượng tuyển'] = None


    try:
        buffer = soup.find('p', string="Hình thức làm việc")
        dict_job['Hình thức làm việc'] = buffer.find_next_sibling('p').get_text() if buffer else None
    except:
        dict_job['Hình thức làm việc'] = None

    try:
        buffer = soup.find('p', string="Độ tuổi")
        dict_job['Độ tuổi'] = buffer.find_next_sibling('p').get_text() if buffer else None
    except:
        dict_job['Độ tuổi'] = None

    try:
        buffer = soup.find('p', string="Yêu cầu bằng cấp")
        dict_job['Yêu cầu bằng cấp'] = buffer.find_next_sibling('p').get_text() if buffer else None
    except:
        dict_job['Yêu cầu bằng cấp'] = None

    try:
        buffer = soup.find('p', string="Yêu cầu kinh nghiệm")
        dict_job['Yêu cầu kinh nghiệm'] = buffer.find_next_sibling('p').get_text() if buffer else None
    except:
        dict_job['Yêu cầu kinh nghiệm'] = None

    try:
        buffer = soup.find('p', string="Ngành nghề")
        dict_job['Ngành nghề'] = buffer.find_next_sibling('p').get_text() if buffer else None
    except:
        dict_job['Ngành nghề'] = None


    try:
        buffer1 = soup.find('div', string="Từ khoá")
        buffer2 = buffer1.find_next_sibling('div')
        keywords = [a.text for a in buffer2.find_all('a')] if buffer2 else None
        dict_job['Từ khóa'] = '; '.join(keywords) if keywords else None
    except:
        dict_job['Từ khóa'] = None

    company = soup.find('div', class_='px-4 md:px-10 py-4 bg-white shadow-sd-12 rounded-sm mt-4 lg:mb-6')

    buffers = company.find_all('div', class_= 'text-14 text-se-neutral-64 w-full max-w-[125px] mr-2')
    try:
        for buffer in buffers:
            if buffer.get_text() == 'Địa chỉ:':
                dict_job['Địa chỉ công ty'] = buffer.find_next_sibling('div').text if buffer else None
            if buffer.get_text() == 'Quy mô:':
                dict_job['Quy mô công ty'] = buffer.find_next_sibling('div').text if buffer else None
    except:
        dict_job['Địa chỉ công ty'] = None
        dict_job['Quy mô công ty'] = None

    job_deatail_list.append(dict_job)

raw_job_details = pd.DataFrame(job_deatail_list)
raw_job_details


## Step 1: Lấy những liên kết của trang web công việc
- Đầu tiên, khi vô một trang web tìm việc, ta sẽ thấy một loạt danh sách các công việc, sau khi nhấn vào từng cái sẽ liên kết trực tiếp đến trang chi tiết công việc. Vậy đầu tiên ta đến trang tìm kiếm và lọc tất cả các đường dẫn đến công việc.

In [ ]:
jobs_list = []

for occupation_id in range(1, occupation_ids + 1):
    for working_method in range(1, working_methods + 1):

        new_url = url.format(occupation_id, working_method)
        print(new_url)

        page_content = get_content(new_url, headers)

        try:
            soup = BeautifulSoup(page_content.text, 'html.parser')
            buffer1 = soup.find('div', class_='relative lg:text-2xl text-xl box-border lg:leading-10 mb-4 m-auto lg:w-full w-[100%] px-0')
            strong_tag = buffer1.find('strong')
            total_jobs = int(strong_tag.text)
        except:
            total_jobs = 0


        print('Total jobs:', total_jobs)

        n_pages = min(51, math.ceil(total_jobs/30) + 1)


        for page in range(1, n_pages + 1):
        # for page in range(1, 2):
            page_url = new_url + f'&page={page}'


            page_content = get_content(page_url, headers)
            try:
                soup = BeautifulSoup(page_content.text, 'html.parser')

                containers = soup.find_all('div', class_='relative lg:h-[115px] w-full flex rounded-sm lg:mb-3 mb-2 lg:hover:shadow-md')
            except:
                containers = []
            for container in containers:
                dict_job = {}

                buffer = BASE_URL + container.find('a', href=True)['href']
                dict_job['Liên kết'] = buffer if buffer else None

                buffer = container.find('div', class_='relative lg:w-full w-11/12 flex items-start flex-1 overflow-hidden pr-2 lg:pr-8')
                dict_job['Tên công việc'] = buffer.text if buffer else None


                buffer = container.find('span', class_='text-se-neutral-80 text-14 whitespace-nowrap font-medium')
                dict_job['Mức lương'] = buffer.text if buffer else None

                buffer = container.find('span', class_='text-se-neutral-80 whitespace-nowrap text-14')
                dict_job['Khu vực tuyển'] = buffer.text if buffer else None

                jobs_list.append(dict_job)
    time.sleep(5)
job_pool = pd.DataFrame(jobs_list)

In [ ]:
# Xóa trung lặp
job_pool = job_pool.drop_duplicates(subset=['Liên kết'])
job_pool.info()

In [ ]:
# Lưu dữ liệu vào file csv
job_pool.drop_duplicates(inplace=True)
job_pool.to_csv(path.job_pool_path, index=False, encoding='utf-8-sig')

# Step 2: Thu thập dữ liệu của từng trang web
- Sau khi thu thập được các link trang web, ta tiến hành thu thập dữ liệu của từng trang web bằng cách sử dụng cách tương tự

In [ ]:
df = pd.read_csv(path.job_pool_path)
# df = job_pd.copy()

In [ ]:
df.info()

In [ ]:

job_deatail_list = []
for i, job_url in enumerate(df['Liên kết']):

    print(f'Processing {i+1}/{len(df)}: {url}')

    job_content = get_content(job_url, headers=headers)
    if (job_content != None):
        soup = BeautifulSoup(job_content.text, 'html.parser')

    # Lưu page content vào file
    with open('job_content.html', 'w') as f:
        f.write(job_content.text)

    dict_job = {}

    dict_job['Liên kết'] = job_url

    overview = soup.find('div', class_='md:ml-7 w-full')
    try:
        buffer = overview.find('h2', class_='font-normal text-16 text-se-neutral-64 mb-4')
        dict_job['Tên công ty'] = buffer.text if buffer else None
    except:
        dict_job['Tên công ty'] = None
    try:
        buffer = overview.find('h1', class_='font-semibold text-18 md:text-24 leading-snug')
        dict_job['Tên công việc'] = buffer.text if buffer else None
    except:
        dict_job['Tên công việc'] = None

    date_view = overview.find_all('span', class_='font-medium pr-1')
    try:
        dict_job['Ngày cập nhật'] = date_view[0].find_next_sibling('span').text if date_view[0] else None
    except:
        dict_job['Ngày cập nhật'] = None

    try:
        dict_job['Lượt xem'] = date_view[1].find_next_sibling('span').text if date_view[1] else None
    except:
        dict_job['Lượt xem'] = None

    try:
        buffer = overview.find('p', class_='font-semibold text-14 text-[#8B5CF6]')
        dict_job['Mức lương'] = buffer.text if buffer else None
    except:
        dict_job['Mức lương'] = None

    try:
        buffer = overview.find_all('a', class_ ="hover:text-se-accent")
        dict_job['Khu vực tuyển'] = ' '.join([element.text for element in buffer]) if buffer else None
    except:
        dict_job['Khu vực tuyển'] = None

    try:
        buffer = soup.find('p', string="Yêu cầu giới tính")
        dict_job['Yêu cầu giới tính'] = buffer.find_next_sibling('p').get_text() if buffer else None
    except:
        dict_job['Yêu cầu giới tính'] = None

    try:
        buffer = soup.find('p', string="Cấp bậc")
        dict_job['Cấp bậc'] = buffer.find_next_sibling('p').get_text() if buffer else None
    except:
        dict_job['Cấp bậc'] = None

    try:
        buffer = soup.find('p', string="Thời gian thử việc")
        dict_job['Thời gian thử việc'] = buffer.find_next_sibling('p').get_text() if buffer else None
    except:
        dict_job['Thời gian thử việc'] = None

    try:
        buffer = soup.find('p', string="Số lượng tuyển")
        dict_job['Số lượng tuyển'] = buffer.find_next_sibling('p').get_text() if buffer else None
    except:
        dict_job['Số lượng tuyển'] = None


    try:
        buffer = soup.find('p', string="Hình thức làm việc")
        dict_job['Hình thức làm việc'] = buffer.find_next_sibling('p').get_text() if buffer else None
    except:
        dict_job['Hình thức làm việc'] = None

    try:
        buffer = soup.find('p', string="Độ tuổi")
        dict_job['Độ tuổi'] = buffer.find_next_sibling('p').get_text() if buffer else None
    except:
        dict_job['Độ tuổi'] = None

    try:
        buffer = soup.find('p', string="Yêu cầu bằng cấp")
        dict_job['Yêu cầu bằng cấp'] = buffer.find_next_sibling('p').get_text() if buffer else None
    except:
        dict_job['Yêu cầu bằng cấp'] = None

    try:
        buffer = soup.find('p', string="Yêu cầu kinh nghiệm")
        dict_job['Yêu cầu kinh nghiệm'] = buffer.find_next_sibling('p').get_text() if buffer else None
    except:
        dict_job['Yêu cầu kinh nghiệm'] = None

    try:
        buffer = soup.find('p', string="Ngành nghề")
        dict_job['Ngành nghề'] = buffer.find_next_sibling('p').get_text() if buffer else None
    except:
        dict_job['Ngành nghề'] = None


    try:
        buffer1 = soup.find('div', string="Từ khoá")
        buffer2 = buffer1.find_next_sibling('div')
        keywords = [a.text for a in buffer2.find_all('a')] if buffer2 else None
        dict_job['Từ khóa'] = '; '.join(keywords) if keywords else None
    except:
        dict_job['Từ khóa'] = None

    company = soup.find('div', class_='px-4 md:px-10 py-4 bg-white shadow-sd-12 rounded-sm mt-4 lg:mb-6')

    buffers = company.find_all('div', class_= 'text-14 text-se-neutral-64 w-full max-w-[125px] mr-2')
    try:
        for buffer in buffers:
            if buffer.get_text() == 'Địa chỉ:':
                dict_job['Địa chỉ công ty'] = buffer.find_next_sibling('div').text if buffer else None
            if buffer.get_text() == 'Quy mô:':
                dict_job['Quy mô công ty'] = buffer.find_next_sibling('div').text if buffer else None
    except:
        dict_job['Địa chỉ công ty'] = None
        dict_job['Quy mô công ty'] = None

    job_deatail_list.append(dict_job)

raw_job_details = pd.DataFrame(job_deatail_list)
raw_job_details.head(5)

In [ ]:
# Lưu vào file craw_dataset.csv
raw_job_details.info()
raw_job_details.to_csv(path.raw_job_details_path, index=False, encoding='utf-8-sig')